# Package installation

In [1]:
%pip install snowballstemmer
%pip install torch
%pip install pandas
%pip install string
%pip install sys
%pip install argparse
%pip install sklearn
%pip install gensim
%pip install keras
%pip install matplotlib
%pip install stanza

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requiremen

# Useful Lib Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from sklearn.utils import shuffle

# Loading the Dataset + Shuffling + Sampling + Setting the Pandas Series for Simplicity

In [3]:
df = pd.read_csv('/Users/thzaamoun/Desktop/Academia/Fall 23/NLP/Assignment 2/VSCODE/ar_reviews_100k.tsv',sep='\t')
reviews = df['text']  # Replace with your actual column name
labels = df['label']    # Replace with your actual label column name
# First, shuffle the DataFrame
df_shuffled = shuffle(df, random_state=42)

#Sample 20% of the dataframe
df_sampled = df_shuffled.sample(frac=1, random_state=42)  # frac=0.2 means 20%

# Then, split the features and labels
X = df_sampled['text'].values
y = df_sampled['label'].values
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


# Text Preprocessing

## Text Cleaning

In [4]:
def clean_text(text):
    # No need to remove diacretics since not there
    # Remove special characters and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

reviews = reviews.apply(clean_text)

# Lemmatization

In [5]:
from snowballstemmer import stemmer

# Arabic stemmer
ar_stemmer = stemmer("arabic")

# Stems all words in given text
def stem_text(text):
    words = text.split()
    stemmed_words = [ar_stemmer.stemWord(word) for word in words]
    return ' '.join(stemmed_words)

# Apply to all entries in the pd Series
reviews = reviews.apply(stem_text)

In [6]:
reviews.head()

0     ممتاز نوع ما نظاف والموقع والتجهيز والشاطيء مطعم
1    احد اسباب نجاح امار ان كل شخص في هذه دول يعشق ...
2    هادف وقو تنقل من صخب شوارع قاهر الي هدوء جبال ...
3    خلص مبديي الل مست ابهار زي فيل ازرق ميقراش احس...
4    ياسا جلوري جزء لا يتجزء من دب ندق متكامل خدم م...
Name: text, dtype: object

## Tokenization

In [7]:
reviews = reviews.apply(lambda x: word_tokenize(x))

# Building a Vocab (PAD + UNK)

In [8]:
from collections import Counter

def build_vocab(sentences, min_word_freq):
    # Count word frequencies
    word_freq = Counter(word for sentence in sentences for word in sentence)
    
    # Remove words below a certain frequency
    word_freq = {word: freq for word, freq in word_freq.items() if freq >= min_word_freq}

    # Build the vocabulary
    vocab = {word: index + 2 for index, word in enumerate(word_freq)} # +2 for special tokens
    vocab['<PAD>'] = 0  # To Pad
    vocab['<UNK>'] = 1  # Add an unknown token

    return vocab

vocab = build_vocab(reviews, min_word_freq=2)

# Text-To-Sequence Conversion

In [9]:
def text_to_sequence(sentences, vocab):
    sequences = []
    for sentence in sentences:
        #UNK for unknown tokens = out-of-vocab
        seq = [vocab.get(word, vocab['<UNK>']) for word in sentence]
        sequences.append(seq)
    return sequences

sequences = text_to_sequence(reviews, vocab)

# Padding Sequences

In [10]:
from keras.preprocessing.sequence import pad_sequences

max_seq_length = 100
sequences_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

2023-11-15 17:12:46.929169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preparing Labels

In [11]:
labels = np.array(labels)

# Splitting Data

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences_padded, labels, test_size=0.2, random_state=42)

In [13]:
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

int32
int32
object
object


# Converting Sequence-Lists to Numpy Arrays

In [15]:
from keras.preprocessing.sequence import pad_sequences

max_seq_length = 100
X_train_padded = pad_sequences(X_train, maxlen=max_seq_length, padding='post', dtype='int64')
X_test_padded = pad_sequences(X_test, maxlen=max_seq_length, padding='post', dtype='int64')

# -> To PyTorch tensors
train_data = torch.tensor(X_train_padded, dtype=torch.long)
test_data = torch.tensor(X_test_padded, dtype=torch.long)

# Encoding Y

In [16]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import torch

# Check for unusual DTs
for item in np.concatenate([y_train.flatten(), y_test.flatten()]):
    if isinstance(item, type(...)) or not isinstance(item, (str, int, float)):
        print("Unusual item found:", item)

# Standardize Ellipses
y_train = np.array(['Unknown' if isinstance(item, type(...)) else item for item in y_train.flatten()])
y_test = np.array(['Unknown' if isinstance(item, type(...)) else item for item in y_test.flatten()])

# Reshape to 2D after replacement
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)

# Fit and transform the training data
y_train_encoded = onehot_encoder.fit_transform(y_train)

# Transform the test data using the same encoder
y_test_encoded = onehot_encoder.transform(y_test)

/Users/thzaamoun/Library/Python/3.11/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Converting to Pytorch Tensors

In [17]:
import torch
train_data = torch.tensor(X_train, dtype=torch.long)
test_data = torch.tensor(X_test, dtype=torch.long)
train_labels = torch.tensor(y_train_encoded, dtype=torch.float32)
test_labels = torch.tensor(y_test_encoded, dtype=torch.float32)

In [18]:
# Check Dimensions of the datasets
print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)
print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)

Train data shape: torch.Size([79999, 100])
Train labels shape: torch.Size([79999, 3])
Test data shape: torch.Size([20000, 100])
Test labels shape: torch.Size([20000, 3])


In [20]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Check shapes
print("train_data shape:", train_data.shape)
print("train_labels shape:", train_labels.shape)
print("test_data shape:", test_data.shape)
print("test_labels shape:", test_labels.shape)

# Correcting mismatch
# Trim train_data to match size of train_labels
if train_data.shape[0] > train_labels.shape[0]:
    train_data = train_data[:train_labels.shape[0]]

# Trim test_data to match the size of test_labels
if test_data.shape[0] > test_labels.shape[0]:
    test_data = test_data[:test_labels.shape[0]]

# Verify the adjustment
print("Adjusted train_data shape:", train_data.shape)
print("Adjusted test_data shape:", test_data.shape)

# Recreate the datasets and loaders
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32)

train_data shape: torch.Size([79999, 100])
train_labels shape: torch.Size([79999, 3])
test_data shape: torch.Size([20000, 100])
test_labels shape: torch.Size([20000, 3])
Adjusted train_data shape: torch.Size([79999, 100])
Adjusted test_data shape: torch.Size([20000, 100])


# Creating DataLoaders for efficient training

In [21]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Loading the Word2Vec

In [22]:
word_vectors = np.load('/Users/thzaamoun/Desktop/Academia/Fall 23/NLP/Assignment 2/VSCODE/full_grams_cbow_300_wiki/full_grams_cbow_300_wiki.mdl.wv.vectors.npy')

# GRU Model Class-Definition

In [28]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_filters, filter_sizes, num_classes, dropout):
        super(CNNModel, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

        # Convolutional layers
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])

        # Fully connected layer
        self.fc = nn.Linear(len(filter_sizes) * num_filters, num_classes)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x = [batch size, sent len]
        x = self.embedding(x)
        # x = [batch size, sent len, emb dim]

        # Convolution and pooling
        x = x.permute(0, 2, 1)  # reshape for conv layers
        # x = [batch size, emb dim, sent len]

        x = [F.relu(conv(x)) for conv in self.convs]  # apply convolution and ReLU
        x = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in x]  # apply max pooling

        # Concatenate the convolutional layer outputs
        x = torch.cat(x, dim=1)

        # Dropout and fully connected layer
        x = self.dropout(x)
        x = self.fc(x)

        return x

# Model Instanciation

In [29]:
num_embeddings = 114000  # Size of your vocabulary
embedding_dim = 300     # Dimension of word embeddings
num_filters = 100       # Number of filters per convolutional layer
filter_sizes = [3, 4, 5]  # Sizes of the convolutional layer filters
num_classes = 3         # Number of output classes
dropout = 0.5           # Dropout rate

model = CNNModel(num_embeddings, embedding_dim, num_filters, filter_sizes, num_classes, dropout)

# Check Integrity of input Data

In [30]:
# Check the maximum index in your input data
max_index = train_data.max()
print("Maximum index in train_data:", max_index)

# Ensure that this max index is less than num_embeddings
assert max_index < model.embedding.num_embeddings, "Index out of range. Increase num_embeddings in your model."

Maximum index in train_data: tensor(57246)


# Mode of Running The Model

In [31]:
device = torch.device('cpu')
model.to(device)

CNNModel(
  (embedding): Embedding(114000, 300)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

# Define Loss Function & Optimizer

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
unique_labels = set()
for _, labels in train_loader:
    labels = torch.argmax(labels, dim=1)
    unique_labels.update(labels.tolist())
print("Unique label indices in the training dataset:", unique_labels)

Unique label indices in the training dataset: {0, 1, 2}


# Training Loop

In [34]:
num_epochs = 10  # Number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        # Convert one-hot encoded labels to class indices if necessary
        if labels.dim() > 1 and labels.size(1) > 1:
            labels = torch.argmax(labels, dim=1)

        # Filter out instances with label '3'
        valid_indices = labels != 3
        inputs = inputs[valid_indices]
        labels = labels[valid_indices]

        # Check if label '3' still exists in the batch
        if labels.max() >= 3:
            print("Error: Label '3' found in batch after filtering")
            continue  # Skip this batch

        optimizer.zero_grad()
        outputs = model(inputs)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
# 3 features for the 3 classes
assert model.fc.out_features == 3, "Model's output layer should have 3 features"

In [37]:
if labels.dim() > 1 and labels.size(1) > 1:
    labels = torch.argmax(labels, dim=1)

# Model Evaluation

In [39]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        # If labels are one-hot encoded or in a 2D format, convert them to 1D class indices
        if labels.dim() > 1 and labels.size(1) > 1:
            labels = torch.argmax(labels, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

Accuracy: 62.505%


# Save The Model

In [40]:
torch.save(model.state_dict(), 'cnn_model_state_dict.pth')